# Přednáška 6: grafické modely, detekce outlierů

## Obsah přednášky

- grafické modely
    - motivace a princip
    - ukázka na nám známé bayesovské lineární regresi
- detekce outlierů
- úvod k datům na cvičení (když bude čas...)

## Grafické modely

Každý bod grafu reprezentuje náhodnou proměnnou a hrany reprezentují statistickou závislost mezi proměnnými. Mějme následující graf:

<img src="img_ot/l7_graf_ABCDE.png" width="400">

Ten lze zapsat ekvivalentně jako
$$
f\left(A,B,C,D,E\right)=f\left(A\right)f\left(B\right)f\left(C|A,B\right)f\left(D|B,C\right)f\left(E|C,D\right)
$$
Formální obecný zápis může vypadat následovně:
$$
f\left(\theta_{1},\dots,\theta_{n}\right)=\prod_{i=1}^{n}f\left(\theta_{i}|\theta_{\text{parents}(i)}\right)
$$

K čemu je to dobré?
- intuitivní reprezentace a vizualizace vztahů a vazeb mezi proměnnými
- princip modularity
- message passing (variační Bayes: pomocí momentů)

My si ukážeme konstrukci grafických modelů na nám již známých modelech bayesovský lineární regrese.

### Bayesovská lineární regrese

$$
f(\mathbf{y}|\beta,\omega)=\mathcal{N}\left(X\beta,\omega^{-1}I_{p}\right),\\
f(\omega)= \mathcal{G}\left(c_{0},d_{0}\right),\\
f(\beta)= \mathcal{N}\left(\mathbf{0},I_{n}\right).
$$

<img src="img_ot/l7_graf_ridge.png" width="300">

### Bayesovská řídká regrese

$$
f(\mathbf{y}|\beta,\omega)=\mathcal{N}\left(X\beta,\omega^{-1}I_{p}\right),\\
f(\omega)= \mathcal{G}\left(c_{0},d_{0}\right),\\
f(\beta|\mathbf{v})=\mathcal{N}\left(\mathbf{0},\text{diag}(\mathbf{v})^{-1}\right),\\
f(v_{j})=\mathcal{G}\left(a_{0},b_{0}\right),\ \forall j.
$$

<img src="img_ot/l7_graf_sparse.png" width="400">

Připomeňme:
\begin{align}
\Sigma_{\beta}= & \left(\widehat{\omega}X^{T}X+\widehat{V}\right)^{-1}\\
\mu_{\beta}= & \Sigma_{\beta}\widehat{\omega}X^{T}\mathbf{y}=\left(\widehat{\omega}X^{T}X+\widehat{V}\right)^{-1}\widehat{\omega}X^{T}\mathbf{y}\\
a_{j}= & a_{0}+\frac{1}{2}\\
b_{j}= & b_{0}+\frac{1}{2}\left(\widehat{\beta\beta^T}\right)_{j,j}\\
c= & c_{0}+\frac{p}{2}\\
d= & d_{0}+\frac{1}{2}\left(\mathbf{y}\mathbf{y}^{T}-2\mathbf{y}^{T}X\widehat{\beta}+\widehat{\beta^{T}X^{T}X\beta}\right)
\end{align}

K vyhodnocení uzlu potřebuju znát
- rodiče
- potomky
- spolu-rodiče

<img src="img_ot/l7_graf_vyhodnocovani.png" width="400">

### Model šumu v baysovské lineární regresi 

<img src="img_ot/l7_graf_sum.png" width="400">



## Detekce outlierů

Co je outlier, resp. odlehlá hodnota, je poměrně spekulativní a často subjektivní (viz chyby s těžkými konci). Pro nás bude stačit, když si řekneme, že je to hodnota měření, jejíž chyba nespadá do námi předpokládaného rozdělení.

Nejprve si na příkladu ukážeme lineární regrese s jedním outlierem ukážeme, jak outlier vypadá a co způsobí. Připomenutí použitých dat:

<img src="img_ot/l5_y_illcond.png"> | = | <img src="img_ot/l4_X_illcond.png"> | <img src="img_ot/l4_beta_true.png">
- | - | - | -
$\mathbf{y}$ | = |<p align="center"> $X$ |<p align="center"> $\beta_{true}$

Řešení bayesovskou řídkou regresí, které jsme již viděli:
<img src="img_ot/l5_beta_VBsparse.png">

Uvažujme teď případ, kdy měření $\mathbf{y}$ obsahuje outlier, tedy nějakou nesmyslnou hodnotu, např. z pokaženého senzoru. V našem případě to budeme simulovat tím, že hodnotu na 13. místě vektoru měření upravíme následovně: $y_{13} = 7*y_{13}$. Data $\mathbf{y}$ pak vypadají následovně:
<img src="img_ot/l7_y_outlier.png">

a odhad pomocí bayesovské řídké regrese takto:
<img src="img_ot/l7_beta_VBsparse_outlier.png">

Co s tím? Stejně jako modelujeme řídkost parametru $\beta$ pomocí tzv. automatic relevance determination (ARD), můžeme obdobný model nasadit do modelu $f(\mathbf{y})$. Kovarianční matici modelu $f(\mathbf{y})$ budeme modelovat jako diagonální s naznámými parametry $\omega_1,\dots,\omega_p$ na diagonále, kterým přiřadíme jakožto apriorní gamma rozdělení.

<img src="img_ot/l7_graf_outliery.png" width="400">

$$
f(\mathbf{y}|\beta,\omega_1,\dots,\omega_p)=\mathcal{N}\left(X\beta,\text{diag}\left(\omega_1,\dots,\omega_p\right)^{-1}\right),\\
f(\omega_i)= \mathcal{G}\left(c_{0},d_{0}\right),\,\forall j\\
f(\beta|\mathbf{v})=\mathcal{N}\left(\mathbf{0},\text{diag}(\mathbf{v})^{-1}\right),\\
f(v_{j})=\mathcal{G}\left(a_{0},b_{0}\right),\,\forall j.
$$

V dalším postupu zavedeme značení $\Omega = \text{diag}\left(\omega_1,\dots,\omega_p\right)$. Příslušné marginální pravděpodobnosti:
$$
\tilde{f}(\beta|\mathbf{y})\propto \exp\left[-\frac{1}{2}(-2)\mathbf{y}^{T}\Omega X\beta-\frac{1}{2}\beta^{T}X^{T}\Omega X\beta-\frac{1}{2}\left(\beta^{T}V\beta\right)\right],\\
\tilde{f}(v_j|\mathbf{y}) \propto \exp\left( (a_{0}-1+\frac{1}{2})\ln v_{j}-\left(b_{0}+\frac{1}{2}\text{diag}\left(\beta\beta^{T}\right)_{j}\right)v_{j} \right),\\
\tilde{f}(\omega_{i}|\mathbf{y})\propto \exp\Bigg[\left(c_{0}+\frac{1}{2}-1\right)\ln\omega_{i} -\left(d_{0}+\frac{1}{2}\text{diag}\left(X\beta\beta^{T}X^{T}\right)_{i,i}-\text{diag}(X\beta\mathbf{y}^{T})_{i,i}+\frac{1}{2}\text{diag}(\mathbf{y}\mathbf{y}^{T})_{i,i}\right)\omega_{i}\Bigg].
$$

Vidíme, že marginála $\tilde{f}(v_j|\mathbf{y})$ je identická se základním modelem s řídkostí, úprava modelu o $\Omega$ k ní "neprobublala". Další marginály identifikujeme jako
$$
\tilde{f}(\beta|\mathbf{y})=\mathcal{N}(\mu_{\beta},\Sigma_{\beta}),\\
\tilde{f}(\omega_i|\mathbf{y})=\mathcal{G}(c_i,d_i),
$$
kde tvarovací parametry $\mu_{\beta},\Sigma_{\beta},c_i,d_i$ jsou
$$
\Sigma_{\beta}= \left(X^{T}\widehat{\Omega}X+\widehat{V}\right)^{-1},\\
\mu_{\beta}=\left(X^{T}\widehat{\Omega}X+\widehat{V}\right)^{-1}X^{T}\widehat{\Omega}\mathbf{y},\\
c_i= c_{0}+\frac{1}{2},\\
d_i= d_{0}+\frac{1}{2}\left(X\widehat{\beta\beta^{T}}X^{T}\right)_{i,i}-(X\widehat{\beta}\mathbf{y}^{T})_{i,i}+\frac{1}{2}(\mathbf{y}\mathbf{y}^{T})_{i,i}.
$$
K tomu poznamenejme moment $\widehat{\Omega}$:
$$
\widehat{\Omega}_{i,i} = \frac{c_i}{d_i}.
$$

Výsledky výpočtu na datech s outlierem a jeho detekce:

<img src="img_ot/l7_beta_VBoutlier.png">

<img src="img_ot/l7_omegas_outlier.png">